In [ ]:
import os
!pip install -q kaggle

if not os.path.exists("/root/.kaggle/kaggle.json"):
    from google.colab import files
    print("Upload your kaggle.json file from your Kaggle account")
    files.upload()
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("kaggle.json already configured")

if not os.path.exists("accepted_2007_to_2018Q4.csv.gz"):
    print("Downloading Lending Club dataset...")
    !kaggle datasets download -d wordsforthewise/lending-club -q
    !unzip -q lending-club.zip
    print("Dataset downloaded & extracted")
else:
    print("Dataset already available")

✅ kaggle.json already configured
✅ Dataset already available


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    classification_report,
    confusion_matrix
)


import warnings

warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("accepted_2007_to_2018Q4.csv.gz", low_memory=False, nrows=200000)

In [ ]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(200000, 151)

In [ ]:
df.info

<bound method DataFrame.info of               id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0       68407277        NaN     3600.0       3600.0           3600.0   
1       68355089        NaN    24700.0      24700.0          24700.0   
2       68341763        NaN    20000.0      20000.0          20000.0   
3       66310712        NaN    35000.0      35000.0          35000.0   
4       68476807        NaN    10400.0      10400.0          10400.0   
...          ...        ...        ...          ...              ...   
199995  56080425        NaN    12000.0      12000.0          12000.0   
199996  55909672        NaN    21000.0      21000.0          21000.0   
199997  54414556        NaN    27500.0      27500.0          27500.0   
199998  56109383        NaN     7000.0       7000.0           7000.0   
199999  56129522        NaN    23000.0      23000.0          23000.0   

              term  int_rate  installment grade sub_grade  ...  \
0        36 months     13.99       123.03     C        C4  ...   
1        36 months     11.99       820.28     C        C1  ...   
2        60 months     10.78       432.66     B        B4  ...   
3        60 months     14.85       829.90     C        C5  ...   
4        60 months     22.45       289.91     F        F1  ...   
...            ...       ...          ...   ...       ...  ...   
199995   36 months     12.69       402.54     C        C2  ...   
199996   36 months     12.29       700.42     C        C1  ...   
199997   60 months     14.65       649.19     C        C5  ...   
199998   36 months     10.99       229.14     B        B4  ...   
199999   60 months     13.99       535.06     C        C4  ...   

       hardship_payoff_balance_amount hardship_last_payment_amount  \
0                                 NaN                          NaN   
1                                 NaN                          NaN   
2                                 NaN                          NaN   
3                                 NaN                          NaN   
4                                 NaN                          NaN   
...                               ...                          ...   
199995                            NaN                          NaN   
199996                            NaN                          NaN   
199997                            NaN                          NaN   
199998                            NaN                          NaN   
199999                            NaN                          NaN   

       disbursement_method  debt_settlement_flag debt_settlement_flag_date  \
0                     Cash                     N                       NaN   
1                     Cash                     N                       NaN   
2                     Cash                     N                       NaN   
3                     Cash                     N                       NaN   
4                     Cash                     N                       NaN   
...                    ...                   ...                       ...   
199995                Cash                     N                       NaN   
199996                Cash                     N                       NaN   
199997                Cash                     N                       NaN   
199998                Cash                     N                       NaN   
199999                Cash                     N                       NaN   

       settlement_status settlement_date settlement_amount  \
0                    NaN             NaN               NaN   
1                    NaN             NaN               NaN   
2                    NaN             NaN               NaN   
3                    NaN             NaN               NaN   
4                    NaN             NaN               NaN   
...                  ...             ...               ...   
199995               NaN             NaN               NaN   
199996               NaN            

In [ ]:
print("Loading data from 'task_4_analysis_data.npz'...")
try:
    data = np.load('task_4_analysis_data.npz', allow_pickle=True)
    y_test = data['y_test']
    rl_policy_actions = data['rl_policy_actions']
    dl_model_probas = data['dl_model_probas']
    reward_if_approve = data['reward_if_approve']
    reward_if_deny = data['reward_if_deny']
    X_test = data['X_test']
    feature_names = data['feature_names']
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'task_4_analysis_data.npz' not found.")
    print("Please run Task 2 and Task 3 successfully to generate this file.")
    exit()

Loading data from 'task_4_analysis_data.npz'...
Data loaded successfully.


In [ ]:
# Policy 1: Historical Policy (Always Approve)
# Action is always 1 (Approve)
historical_rewards = reward_if_approve
historical_total_profit = np.sum(historical_rewards)
historical_avg_profit = np.mean(historical_rewards)
historical_approval_rate = 1.0

# Policy 2: DL Model Policy (Approve if prob(default) < 0.5)
# Action = 0 (Deny) if proba > 0.5, else 1 (Approve)
dl_policy_actions = (dl_model_probas <= 0.5).astype(int)
dl_rewards = np.where(
    dl_policy_actions == 1,
    reward_if_approve,
    reward_if_deny
)
dl_total_profit = np.sum(dl_rewards)
dl_avg_profit = np.mean(dl_rewards)
dl_approval_rate = np.mean(dl_policy_actions)

# Policy 3: RL Agent Policy (Learned by CQL)
# Actions are directly from rl_policy_actions
rl_rewards = np.where(
    rl_policy_actions == 1,
    reward_if_approve,
    reward_if_deny
)
rl_total_profit = np.sum(rl_rewards)
rl_avg_profit = np.mean(rl_rewards)
rl_approval_rate = np.mean(rl_policy_actions)

In [ ]:
print("\n--- Final Policy Comparison (Business Metrics) ---")

results = {
    "Policy": ["1. Historical (Always Approve)", "2. DL Model (Threshold=0.5)", "3. RL Agent (Profit-Max)"],
    "Total Profit on Test Set": [f"${historical_total_profit:,.2f}", f"${dl_total_profit:,.2f}", f"${rl_total_profit:,.2f}"],
    "Average Profit per Loan": [f"${historical_avg_profit:,.2f}", f"${dl_avg_profit:,.2f}", f"${rl_avg_profit:,.2f}"],
    "Approval Rate": [f"{historical_approval_rate*100:.2f}%", f"{dl_approval_rate*100:.2f}%", f"{rl_approval_rate*100:.2f}%"]
}
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))


print("\n--- Policy Disagreement Analysis ---")

# Find cases where the models disagree
df = pd.DataFrame()
df['dl_action'] = dl_policy_actions
df['rl_action'] = rl_policy_actions
df['true_outcome'] = y_test # 0=Paid, 1=Default
df['reward_if_approve'] = reward_if_approve

# Case 1: DL Denies, RL Approves
dl_denies_rl_approves = df[
    (df['dl_action'] == 0) & (df['rl_action'] == 1)
]

print(f"\nFound {len(dl_denies_rl_approves)} cases where DL Denied but RL Approved.")
if len(dl_denies_rl_approves) > 0:
    # Analyze these cases: What was the *actual* outcome?
    print("Outcomes of these cases:")
    print(dl_denies_rl_approves['true_outcome'].value_counts(normalize=True))
    print(f"Average reward from these 'missed opportunities': ${dl_denies_rl_approves['reward_if_approve'].mean():.2f}")


# Case 2: DL Approves, RL Denies
dl_approves_rl_denies = df[
    (df['dl_action'] == 1) & (df['rl_action'] == 0)
]

print(f"\nFound {len(dl_approves_rl_denies)} cases where DL Approved but RL Denied.")
if len(dl_approves_rl_denies) > 0:
    # Analyze these cases: What was the *actual* outcome?
    print("Outcomes of these cases:")
    print(dl_approves_rl_denies['true_outcome'].value_counts(normalize=True))
    print(f"Average loss avoided by RL in these cases: ${dl_approves_rl_denies['reward_if_approve'].mean():.2f}")


--- Final Policy Comparison (Business Metrics) ---
                        Policy Total Profit on Test Set Average Profit per Loan Approval Rate
1. Historical (Always Approve)          $-63,972,634.52              $-1,816.53       100.00%
   2. DL Model (Threshold=0.5)           $-6,569,940.06                $-186.56        59.00%
      3. RL Agent (Profit-Max)             $-424,549.21                 $-12.06         1.15%

--- Policy Disagreement Analysis ---

Found 309 cases where DL Denied but RL Approved.
Outcomes of these cases:
true_outcome
0    0.699029
1    0.300971
Name: proportion, dtype: float64
Average reward from these 'missed opportunities': $-1460.86

Found 20681 cases where DL Approved but RL Denied.
Outcomes of these cases:
true_outcome
0    0.892703
1    0.107297
Name: proportion, dtype: float64
Average loss avoided by RL in these cases: $-318.98
